In [ ]:
# Install dependencies for phylogenetic analysis 
conda create -n phylo_te -c bioconda -c conda-forge roary iqtree mafft fasttree prokka

conda activate phylo_te

# Verify installation of dependencies 
roary -v
iqtree -v
mafft --version
fasttree -help
prokka --version

### Roary para analise de pangenoma e geracao de alinhamento.

### IQ-TREE para construcao da arvore filogenetica.

### iTOL para visualizacao da arvore e anotacao das IS.

In [ ]:
# 1. anotar genes
prokka --outdir GCF_001_prokka --prefix GCF_001 GCF_001.fasta

## --outdir eh o diretorio de saida, ou seja, onde serao armazenados os arquivos de saida
## --prefix eh o prefixo dos arquivos de saida, ou seja, o nome que sera dado aos arquivos
 
## NCBI.fasta eh o arquivo de entrada, ou seja, o arquivo que sera anotado, que e o arquivo do genoma do NCBI

### Fazer esta etapa para cada genoma que for ser analisado, ou seja, para cada genoma do NCBI

# MODO AUTOMATICO #
## Abra o nano ##
for file in *.fasta; do
    base=$(basename "$file" .fasta)
    prokka --outdir "${base}_prokka" --prefix "$base" "$file"
done
### copiar e colar o comando acima 
#### Aperte Ctrl + o para salvar, salvar como run_prokka.sh, aperte Enter para salvar
##### Aperte Ctrl + x para sair do nano
###### Agora e so rodar o script ---> bash run_prokka.sh

In [ ]:
# criar um diretorio para a coleção de arquivos .gff gerados
cd /mnt/dados/victor_baca/outputs/Phylogenetic_analysis/iq-tree
mkdir -p gff_collection

# copiar os arquivos .gff gerados no diretorio gff_collection
for diretorio in *_prokka; do
    cp "$diretorio"/*.gff /mnt/dados/victor_baca/outputs/Phylogenetic_analysis/iq-tree/gff_collection/
done 

## 2. rodar o roary
roary -e -n -v -f roary_output *.gff
### -e: usa alinhamento de nucleotideos 
### -n: nao filtra para apenas um representante por gene 
### -v: modo verboso 
### -f: pasta de saida
## coloca o gff do prokka fez
### vai gerar um arquivo .csv e um .aln

In [ ]:
### 3. rodar o iq-tree
cd /mnt/dados/victor_baca/outputs/Phylogenetic_analysis/iq-tree/gff_collection/roary_output
iqtree -s core_gene_alignment.aln -nt AUTO -m GTR+G -bb 1000
###vai criar um arquivo core_gene_alignment.aln.treefile

In [ ]:
# 4. criar um novo arquivo .tsv com os aarquivos .tsv do ISEScan
''' genome  IS3     IS256   IS30
    GCF_001 2       0       1
    GCF_002 1       1       0
    GCF_003 3       2       1'''

# SCRIPT PARA GERAR O ARQUIVO .TSV COM AS IS E AS CEPAS ###########################################################
## Para utilizar este script, é necessário ter os arquivos .tsv do ISEScan numa pasta unica.######################
### Ou seja, todos os arquivos .tsv do ISEScan devem estar na mesma pasta.########################################
#### Codigo para fazer a copia de todos os arquivos .tsv de um determinado diretorio para outro (sem importar se esta em subdiretorios)
find '/mnt/dados/victor_baca/outputs/isescan' -type f -name "*.tsv" -exec cp -t '/mnt/dados/victor_baca/outputs/Phylogenetic_analysis/iq-tree/tsv' {} +
##### Para rodar o script, pois a variavel "pasta_tsv" deve ser alterada para o caminho da pasta onde estão os arquivos .tsv do ISEScan.##########
import csv
import os
from collections import defaultdict

# Pasta onde estão os arquivos .tsv do ISEScan
pasta_tsv = "/mnt/dados/victor_baca/outputs/Phylogenetic_analysis/iq-tree/tsv"

# Inicializa estrutura para armazenar dados
dados_gerais = defaultdict(lambda: defaultdict(int))
familias_todas = set()

# Percorre todos os arquivos .tsv
for arquivo in os.listdir(pasta_tsv):
    if not arquivo.endswith(".tsv"):
        continue

    caminho = os.path.join(pasta_tsv, arquivo)
    cepa = os.path.splitext(arquivo)[0]  # ex: strepto1_isescan → strepto1_isescan

    # Lê o arquivo
    with open(caminho) as f:
        reader = csv.DictReader(f, delimiter='\t')
        for linha in reader:
            familia = linha['family']
            dados_gerais[cepa][familia] += 1
            familias_todas.add(familia)

# Ordena as famílias
familias_ordenadas = sorted(familias_todas)

# Escreve arquivo de saída
with open("is_counts.tsv", "w", newline='') as f_out:
    writer = csv.writer(f_out, delimiter='\t')
    writer.writerow(["genome"] + familias_ordenadas)

    for cepa in sorted(dados_gerais):
        linha = [cepa]
        for fam in familias_ordenadas:
            linha.append(dados_gerais[cepa].get(fam, 0))
        writer.writerow(linha)

print("Arquivo 'is_counts.tsv' gerado com sucesso!")

############################################################## output -> is_counts.tsv

# Apos isso transforme o arquivo tsv gerado pelo script em um arquivo txt, com o script abaixo
################################################################################################
import csv

input_file = "is_counts.tsv"
output_file = "itol_is_binary.txt"

# Parâmetros visuais
shape = "2"  # quadrado
cores_padrao = ["#e6194b", "#3cb44b", "#ffe119", "#4363d8", "#f58231", "#911eb4", "#46f0f0", "#f032e6", "#bcf60c", "#fabebe"]

with open(input_file) as f:
    reader = csv.reader(f, delimiter='\t')
    header = next(reader)
    familias = header[1:]  # Ignora 'genome'
    
    # Gera cores (repete se faltar)
    cores = (cores_padrao * ((len(familias) // len(cores_padrao)) + 1))[:len(familias)]
    formas = [shape] * len(familias)
    
    dados = list(reader)

with open(output_file, 'w', newline='') as out:
    out.write("DATASET_BINARY\n")
    out.write("SEPARATOR TAB\n")
    out.write("DATASET_LABEL\tIS elements\n")
    out.write("FIELD_LABELS\t" + "\t".join(familias) + "\n")
    out.write("FIELD_COLORS\t" + "\t".join(cores) + "\n")
    out.write("FIELD_SHAPES\t" + "\t".join(formas) + "\n")
    out.write("DATA\n")

    for linha in dados:
        out.write("\t".join(linha) + "\n")

################################################################################### output -> itol_is_binary.txt


In [ ]:
#### 5. Visualizar no iTOL
## Va para https://itol.embl.de
## Clique em “Upload Tree”
## Suba o arquivo: core_gene_alignment.aln.treefile
## Clique em “Datasets” > “Upload” > selecione o .txt com ISs do arquivo .tsv (itol_is_binary.txt)
## Visualize com circulos coloridos ou heatmap por IS

In [ ]:
# Copiar um arquivo do servidor para maquina local (windowns)
scp victorbaca@10.20.34.31:/mnt/dados/victor_baca/outputs/Phylogenetic_analysis/iq-tree/tsv/itol_is_binary.txt "C:\Users\vitor\Downloads"